### [01] 구매 데이터를 이용하여 송장 파일 만들기

In [ ]:
# 기본 송장 템플릿 읽기
from docx import Document

# 워드 파일 열기
doc = Document('송장.docx')

# 단락 읽기 
for index, paragraph in enumerate(doc.paragraphs):
  print(f'단락 번호 {index}:') # 단락 인덱스 출력

  # 각 단락 내에서 개별 텍스트 런의 서식을 확인합니다.
  for index, run in enumerate(paragraph.runs):
    print(f'  텍스트 번호: {index}')
    print(f'  텍스트 내용: {run.text}')

# 표 확인하기
# 문서에 있는 모든 표를 순회
for table_index, table in enumerate(doc.tables):
  print(f'표 {table_index}:') #표의 인덱스 출력

  # 표의 각 행을 순회
  for row_index, row in enumerate(table.rows):
    print(f'  행 {row_index}:') # 행 번호 출력

    # 행의 각 셀을 순회
    for cell_index, cell in enumerate(row.cells):
      print(f'  셀 {cell_index}: {cell.text}') # 셀의 인덱스 출력

  print('-' * 20)  # 단락 간 구분선

In [ ]:
# 템플릿에 데이터 입력하기
from docx import Document
import os

# 워드 문서 불러오기
doc = Document('송장.docx')

# ➊ 고객 정보를 입력해야 하는 단락 지정
target_paragraph = doc.paragraphs[2]

# ➋ 각 라벨 뒤에 텍스트 추가
target_paragraph.runs[2].add_text('이호연')
target_paragraph.runs[5].add_text('서울시 마포구 서교동')
target_paragraph.runs[8].add_text('070-333-4444')

# ➌ 구매 내역을 입력할 표 지정
table = doc.tables[0]

# ➍ 표에 입력할 데이터를 'data' 변수에 리스트로 저장
data = [
 ['제품1', '2', '12,000', '24,000']
]

# ➎ 헤더를 제외한 표의 두 번째 행부터 각 셀에 data를 입력
for i, row_data in enumerate(data):
  for j, cell_data in enumerate(row_data):
    table.cell(i+1, j).text = cell_data

# ➏ 표의 마지막 행의 마지막 셀에 '총합계'를 입력
table.rows[-1].cells[-1].text = '240,000'

# 생성된 파일을 저장할 '송장폴더'가 존재하지 않는 경우 생성
if not os.path.exists('./송장폴더'):
    os.makedirs('./송장폴더')

# 문서 저장
doc.save('./송장폴더/이호연_송장.docx')

### [02] 여러 파일 한 번에 작성하기

In [ ]:
from openpyxl import load_workbook
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os

# 엑셀 파일 불러오기
wb = load_workbook('구매고객리스트.xlsx', data_only = True)
ws = wb.active

# ➊ 엑셀 파일의 각 행에 대해 워드 파일 생성
for row_idx, row in enumerate(ws.iter_rows(values_only=True)):
  # ➋ 헤더 행은 건너뛰기
  if row_idx == 0:
    continue

  # ➌ 데이터가 없는 행을 건너뛰기
  if not any(row):
    continue

  # ➍ 워드 템플릿 파일 불러오기
  doc = Document('송장.docx')

  # ➎ 고객 정보를 입력해야 하는 단락 지정
  target_paragraph = doc.paragraphs[2]

  # ➏ 각 라벨 뒤에 텍스트 추가
  target_paragraph.runs[2].add_text(str(row[0]))  # 고객명
  target_paragraph.runs[5].add_text(str(row[1]))  # 고객 주소
  target_paragraph.runs[8].add_text(str(row[2]))  # 고객 연락처

  # ➐ 구매 내역을 입력할 표 지정
  table = doc.tables[0]

  # ➑ 표에 입력할 데이터 설정
  data = [[str(row[3]), str(int(row[4])), f'{int(row[5]) if row[5] is not None else 0:,}', f'{int(row[6]) if row[6] is not None else 0:,}']]

  # ➒ 헤더를 제외한 표의 두 번째 행부터 각 셀에 data를 입력
  for i, row_data in enumerate(data):
    for j, cell_data in enumerate(row_data):
      cell = table.cell(i+1, j)
      cell.text = cell_data
      for paragraph in cell.paragraphs:
        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  # ➓ 표의 마지막 행과 셀에 '총합계' 값을 입력
  table.rows[-1].cells[-1].text = f'{int(row[6]) if row[6] is not None else 0:,}'
  for paragraph in table.rows[-1].cells[-1].paragraphs:
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  # 생성된 파일을 저장할 '송장폴더'가 존재하지 않는 경우 생성
  if not os.path.exists('./송장폴더'):
    os.makedirs('./송장폴더')

  # 문서 저장(고객명 row[0]을 파일명에 사용)
  doc.save(f'./송장폴더/{row[0]}_송장.docx')